In [101]:
import requests
import os
import sys
import pandas as pd
import numpy as np

In [102]:
url = "https://db.ygoprodeck.com/api/v7/cardinfo.php"

response = requests.get(url)

if response.status_code != 200:
    sys.exit("Could not fetch card data")

In [103]:
data = response.json()["data"]

In [104]:
def parse_card(card_info):
    parsed = {}
    parsed["id"] = card_info["id"]
    parsed["name"] = card_info["name"]
    parsed["type"] = card_info["type"]
    parsed["frame"] = card_info["frameType"]
    parsed["card_text"] = card_info["desc"]
    parsed["tcgplayer_price_usd"] = card_info["card_prices"][0]["tcgplayer_price"]
    parsed["cardmarket_price_eur"] = card_info["card_prices"][0]["cardmarket_price"]
    parsed["num_alternative_arts"] = len(card_info["card_images"]) - 1 # One of the card images is the original
    parsed["image_url"] = card_info["card_images"][0]["image_url"]

    if "card_sets" in card_info:
        parsed["num_reprints"] = len(card_info["card_sets"]) - 1 # One of the prints is not a reprint

    if "archetype" in card_info:
        parsed["archetype"] = card_info["archetype"]
    
    if "monster" in parsed["type"].lower():
        parsed["atk"] = card_info["atk"]
        parsed["level"] = card_info["level"] if "level" in card_info else card_info["linkval"]
        parsed["attribute"] = card_info["attribute"]
        parsed["monster_type"] = card_info["race"]

        if "pendulum" in parsed["type"].lower():
            parsed["scale"] = card_info["scale"]
        
        if "link" in parsed["type"].lower():
            parsed["link_markers"] = card_info["linkmarkers"]
        else:
            parsed["def"] = card_info["def"]

    
    elif "spell" in parsed["type"].lower():
        parsed["spell_type"] = card_info["race"]
    
    elif "trap" in parsed["type"].lower():
        parsed["trap_type"] = card_info["race"]
    
    return parsed




In [105]:
parsed_data = [parse_card(card) for card in data]

In [106]:
df = pd.DataFrame(parsed_data)
df = df.set_index("id")

In [107]:
df.to_csv(os.path.join("data", "cards.csv"))

In [108]:
df

,name,type,frame,card_text,tcgplayer_price_usd,cardmarket_price_eur,num_alternative_arts,image_url,num_reprints,archetype,spell_type,atk,level,attribute,monster_type,def,trap_type,scale,link_markers
id,,,,,,,,,,,,,,,,,,,
34541863,"""A"" Cell Breeding Device",Spell Card,spell,"During each of your Standby Phases, put 1 A-Co...",0.19,0.17,0,https://images.ygoprodeck.com/images/cards/345...,0.0,Alien,Continuous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64163367,"""A"" Cell Incubator",Spell Card,spell,Each time an A-Counter(s) is removed from play...,0.10,0.23,0,https://images.ygoprodeck.com/images/cards/641...,0.0,Alien,Continuous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91231901,"""A"" Cell Recombination Device",Spell Card,spell,Target 1 face-up monster on the field; send 1 ...,0.21,0.11,0,https://images.ygoprodeck.com/images/cards/912...,0.0,Alien,Quick-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73262676,"""A"" Cell Scatter Burst",Spell Card,spell,"Select 1 face-up ""Alien"" monster you control. ...",0.11,0.13,0,https://images.ygoprodeck.com/images/cards/732...,0.0,Alien,Quick-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37478723,"""Infernoble Arms - Durendal""",Spell Card,spell,While this card is equipped to a monster: You ...,0.25,0.22,0,https://images.ygoprodeck.com/images/cards/374...,2.0,Noble Knight,Equip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2648201,ZW - Sleipnir Mail,Effect Monster,effect,"You can target 1 ""Utopia"" monster you control;...",0.26,0.12,0,https://images.ygoprodeck.com/images/cards/264...,0.0,Utopia,NaN,1000.0,4.0,LIGHT,Beast,1000.0,NaN,NaN,NaN
95886782,ZW - Sylphid Wing,Effect Monster,effect,"You can only control 1 ""ZW - Sylphid Wing"". Yo...",0.10,0.06,0,https://images.ygoprodeck.com/images/cards/958...,0.0,Utopia,NaN,800.0,4.0,LIGHT,Beast,1600.0,NaN,NaN,NaN
81471108,ZW - Tornado Bringer,Effect Monster,effect,"You can target 1 ""Utopia"" monster you control;...",0.11,0.16,0,https://images.ygoprodeck.com/images/cards/814...,1.0,Utopia,NaN,1300.0,5.0,WIND,Dragon,1800.0,NaN,NaN,NaN


In [109]:
df = df[
    [
        "name",
        "type",
        "archetype",
        "tcgplayer_price_usd",
        "cardmarket_price_eur",
        "num_reprints",
        "num_alternative_arts",
        "monster_type",
        "spell_type",
        "trap_type",
        "atk",
        "def",
        "level",
        "attribute",
        "scale",
        "link_markers",
        "frame",
        "image_url",
        "card_text",
    ]
]
df

,name,type,archetype,tcgplayer_price_usd,cardmarket_price_eur,num_reprints,num_alternative_arts,monster_type,spell_type,trap_type,atk,def,level,attribute,scale,link_markers,frame,image_url,card_text
id,,,,,,,,,,,,,,,,,,,
34541863,"""A"" Cell Breeding Device",Spell Card,Alien,0.19,0.17,0.0,0,NaN,Continuous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spell,https://images.ygoprodeck.com/images/cards/345...,"During each of your Standby Phases, put 1 A-Co..."
64163367,"""A"" Cell Incubator",Spell Card,Alien,0.10,0.23,0.0,0,NaN,Continuous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spell,https://images.ygoprodeck.com/images/cards/641...,Each time an A-Counter(s) is removed from play...
91231901,"""A"" Cell Recombination Device",Spell Card,Alien,0.21,0.11,0.0,0,NaN,Quick-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spell,https://images.ygoprodeck.com/images/cards/912...,Target 1 face-up monster on the field; send 1 ...
73262676,"""A"" Cell Scatter Burst",Spell Card,Alien,0.11,0.13,0.0,0,NaN,Quick-Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spell,https://images.ygoprodeck.com/images/cards/732...,"Select 1 face-up ""Alien"" monster you control. ..."
37478723,"""Infernoble Arms - Durendal""",Spell Card,Noble Knight,0.25,0.22,2.0,0,NaN,Equip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spell,https://images.ygoprodeck.com/images/cards/374...,While this card is equipped to a monster: You ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2648201,ZW - Sleipnir Mail,Effect Monster,Utopia,0.26,0.12,0.0,0,Beast,NaN,NaN,1000.0,1000.0,4.0,LIGHT,NaN,NaN,effect,https://images.ygoprodeck.com/images/cards/264...,"You can target 1 ""Utopia"" monster you control;..."
95886782,ZW - Sylphid Wing,Effect Monster,Utopia,0.10,0.06,0.0,0,Beast,NaN,NaN,800.0,1600.0,4.0,LIGHT,NaN,NaN,effect,https://images.ygoprodeck.com/images/cards/958...,"You can only control 1 ""ZW - Sylphid Wing"". Yo..."
81471108,ZW - Tornado Bringer,Effect Monster,Utopia,0.11,0.16,1.0,0,Dragon,NaN,NaN,1300.0,1800.0,5.0,WIND,NaN,NaN,effect,https://images.ygoprodeck.com/images/cards/814...,"You can target 1 ""Utopia"" monster you control;..."
